In [2]:
# imports
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio import Entrez

from IPython.display import Image

from wand.image import Image as wandImage

## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

In [18]:
# get all 'pathway's from organism 'mmu' (Mus musculus)
pathwaysResponse = kegg_list('pathway', 'mmu').read()
print(pathwaysResponse)

path:mmu00010	Glycolysis / Gluconeogenesis - Mus musculus (mouse)
path:mmu00020	Citrate cycle (TCA cycle) - Mus musculus (mouse)
path:mmu00030	Pentose phosphate pathway - Mus musculus (mouse)
path:mmu00040	Pentose and glucuronate interconversions - Mus musculus (mouse)
path:mmu00051	Fructose and mannose metabolism - Mus musculus (mouse)
path:mmu00052	Galactose metabolism - Mus musculus (mouse)
path:mmu00053	Ascorbate and aldarate metabolism - Mus musculus (mouse)
path:mmu00061	Fatty acid biosynthesis - Mus musculus (mouse)
path:mmu00062	Fatty acid elongation - Mus musculus (mouse)
path:mmu00071	Fatty acid degradation - Mus musculus (mouse)
path:mmu00072	Synthesis and degradation of ketone bodies - Mus musculus (mouse)
path:mmu00100	Steroid biosynthesis - Mus musculus (mouse)
path:mmu00120	Primary bile acid biosynthesis - Mus musculus (mouse)
path:mmu00130	Ubiquinone and other terpenoid-quinone biosynthesis - Mus musculus (mouse)
path:mmu00140	Steroid hormone biosynthesis - Mus musculus

In [19]:
# extract pathway IDs from the raw REST response
pathways = [((pathway.split('\t')[0]).split(':'))[-1] for pathway in pathwaysResponse.split('\n')[:-1]]
# print the number of pathways and the IDs of the first 10
print(len(pathways))
pathways[:10]

326


['mmu00010',
 'mmu00020',
 'mmu00030',
 'mmu00040',
 'mmu00051',
 'mmu00052',
 'mmu00053',
 'mmu00061',
 'mmu00062',
 'mmu00071']

In [14]:
# for each pathway, get all contained genes from KEGG (in a similar format as the list of pathways above)
# and transform that raw response into a list of gene IDs for each pathway.
# these lists then should be stored in a dictionary with the pathway IDs as keys

# HINT: use kegg_link(organism, pathway).read()
exampleGeneList = kegg_link('mmu', pathways[0]).read()
print(exampleGeneList)

### Subtask 1.2: Transform the gene ids to a format that can be used with your DE analysis

In [7]:
# Use this function (that uses the Entrez REST service) to transform KEGG IDs (like 'mmu:104112')
# to the type of gene symbols, that we also used for the DE analysis (like 'Gapdh-ps15')

# Enter an email address here, e.g. your student address.
# Entrez needs one in case you flood their servers with requests and they want to block you
# (which should not happen with the few thousand requests we do here)

#Entrez.email = "some.name@student.uni-tuebingen.de"

def entrez2symbol(entrezList):
    request = Entrez.epost("gene", id = ",".join(entrezList))
    try:
        result = Entrez.read(request)
    except RuntimeError as e:
        #FIXME: How generate NAs instead of causing an error with invalid IDs?
        print("An error occurred while retrieving the annotations.")
        print("The error returned was %s" % e)
        return []
    webEnv = result["WebEnv"]
    queryKey = result["QueryKey"]
    data = Entrez.esummary(db = "gene", webenv = webEnv, query_key = queryKey)
    annotations = Entrez.read(data)
    symbolList =[]
    for k in range(len(entrezList)):
        if annotations[u'DocumentSummarySet'][u'DocumentSummary'][k][u'NomenclatureSymbol'] == '':
            symbolList.append(entrezList[k])
        else:
            symbolList.append(annotations[u'DocumentSummarySet'][u'DocumentSummary'][k][u'NomenclatureSymbol'])
            
    return symbolList

### Optional Subtask 1.3: If you have too much time, save the KEGG gene sets as a gmt file <br>
we will not need it for further analysis, but gmt is a commonly used format for gene sets. It would be very useful if you want to rerun this notebook in the future, without having to rerun the calls to the KEGG REST service in 1.1
or the Entrez service in 1.2

hints: 

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)

In [13]:
# some hints on how the data structures of a random pathway look like
pathway = KGML_parser.read(kegg_get('mmu00010', "kgml"))
print('### Pathway description ###')
print(pathway)

for gene in pathway.genes[0:3]:
    print('### New gene: ###')
    print(gene.name)
    for graphic in gene.graphics:
        print('### Current gene color: ###')
        print(graphic.bgcolor)
        
        
# use KGMLCanvas(...) to draw PDFs
# and wandImage(filename = "your.pdf") to display here
# (see imports to find out what wandImage is)

### Pathway description ###
Pathway: Glycolysis / Gluconeogenesis
KEGG ID: path:mmu00010
Image file: http://www.kegg.jp/kegg/pathway/mmu/mmu00010.png
Organism: mmu
Entries: 100
Entry types:
	ortholog: 27
	gene: 35
	compound: 31
	map: 7

### New gene: ###
mmu:11674 mmu:11676 mmu:230163 mmu:353204
### Current gene color: ###
#BFFFBF
### New gene: ###
mmu:110695 mmu:11669 mmu:11671 mmu:56752 mmu:72535
### Current gene color: ###
#BFFFBF
### New gene: ###
mmu:11670 mmu:56847 mmu:621603 mmu:67689 mmu:73458
### Current gene color: ###
#BFFFBF
